In [8]:
from torch.utils.data import Dataset
import torchvision
import torch

import pandas as pd
import numpy as np

In [15]:
# Creating an array (x) with one dimension
x = np.random.randint(0, 255, (1,)) 

# Converting the array (x) to PyTorch tensor using ToTensor
x_tensor = torchvision.transforms.ToTensor()(x)
x_tensor

ValueError: pic should be 2/3 dimensional. Got 1 dimensions.

In [6]:
def make_dataset_rolling(path_to_data):
    data = pd.read_csv(path_to_data)
    value_array = np.array(data.iloc[:,1])
    res = []
    
    H = 5
    h = 1
    
    for i in range(len(data)-H):
        sub_array = value_array[i:i+H]
        res.append((torch.from_numpy(sub_array[:-h]), torch.from_numpy(sub_array[-h:])))
    
    return res 

class CustomImageDataset(Dataset):
    def __init__(self, path_to_data, transform=None, target_transform=None):
        self.roll_data = make_dataset_rolling(path_to_data=path_to_data)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.roll_data)

    def __getitem__(self, idx):
        data, label = self.roll_data[idx]
        if self.transform:
            data = self.transform(data)
        if self.target_transform:
            label = self.target_transform(label)
        return data, label

In [7]:
PATH = 'data/wind_SUD.csv'

transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
dataset = CustomImageDataset(path_to_data=PATH)
dataset[0]

(tensor([42.5000, 50.2000, 45.4000, 29.2000], dtype=torch.float64),
 tensor([4.8000], dtype=torch.float64))